In [1]:
# from google.colab import userdata
# import os

# os.environ["KAGGLE_KEY"] = userdata.get('KAGGLE_KEY')
# os.environ["KAGGLE_USERNAME"] = userdata.get('KAGGLE_USERNAME')


In [2]:
# !kaggle datasets download -d aayush9753/image-colorization-dataset
# ! unzip "image-colorization-dataset.zip"
# !pip install gradio

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  inflating: data/train_color/image0000.jpg  
  inflating: data/train_color/image0001.jpg  
  inflating: data/train_color/image0002.jpg  
  inflating: data/train_color/image0003.jpg  
  inflating: data/train_color/image0004.jpg  
  inflating: data/train_color/image0005.jpg  
  inflating: data/train_color/image0006.jpg  
  inflating: data/train_color/image0007.jpg  
  inflating: data/train_color/image0008.jpg  
  inflating: data/train_color/image0009.jpg  
  inflating: data/train_color/image0010.jpg  
  inflating: data/train_color/image0011.jpg  
  inflating: data/train_color/image0012.jpg  
  inflating: data/train_color/image0013.jpg  
  inflating: data/train_color/image0014.jpg  
  inflating: data/train_color/image0015.jpg  
  inflating: data/train_color/image0016.jpg  
  inflating: data/train_color/image0017.jpg  
  inflating: data/train_color/image0018.jpg  
  inflating: data/train_color/image0019.jpg  
  inflating: data/train

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Concatenate, Flatten, Dense, LeakyReLU, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import cv2
import os

# Enable GPU usage
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
if gpu_devices:
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print("GPU is available and being used.")
else:
    print("GPU is not available, using CPU.")

def build_generator(input_shape=(256, 256, 1)):
    inputs = Input(shape=input_shape)

    # Encoder
    c1 = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    p1 = MaxPooling2D((2, 2))(c1)

    c2 = Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    p2 = MaxPooling2D((2, 2))(c2)

    c3 = Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    p3 = MaxPooling2D((2, 2))(c3)

    c4 = Conv2D(512, (3, 3), activation='relu', padding='same')(p3)

    # Decoder
    u1 = UpSampling2D((2, 2))(c4)
    m1 = Concatenate()([u1, c3])
    c5 = Conv2D(256, (3, 3), activation='relu', padding='same')(m1)

    u2 = UpSampling2D((2, 2))(c5)
    m2 = Concatenate()([u2, c2])
    c6 = Conv2D(128, (3, 3), activation='relu', padding='same')(m2)

    u3 = UpSampling2D((2, 2))(c6)
    m3 = Concatenate()([u3, c1])
    c7 = Conv2D(64, (3, 3), activation='relu', padding='same')(m3)

    outputs = Conv2D(2, (3, 3), activation='tanh', padding='same')(c7)
    model = Model(inputs, outputs)
    return model

def build_discriminator(input_shape=(256, 256, 2)):
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), strides=2, padding='same')(inputs)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(128, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Conv2D(256, (3, 3), strides=2, padding='same')(x)
    x = BatchNormalization()(x)
    x = LeakyReLU(alpha=0.2)(x)

    x = Flatten()(x)
    x = Dense(1, activation='sigmoid')(x)
    model = Model(inputs, x)
    return model

def load_dataset(black_dir, color_dir, img_size=(256, 256)):
    X_train, Y_train = [], []
    filenames = os.listdir(black_dir)

    for filename in filenames:
        black_path = os.path.join(black_dir, filename)
        color_path = os.path.join(color_dir, filename)

        if os.path.exists(color_path):
            black_img = cv2.imread(black_path, cv2.IMREAD_GRAYSCALE)
            color_img = cv2.imread(color_path)

            black_img = cv2.resize(black_img, img_size) / 255.0
            color_img = cv2.resize(color_img, img_size)

            color_lab = cv2.cvtColor(color_img, cv2.COLOR_BGR2LAB)

            X_train.append(black_img[:, :, np.newaxis])
            Y_train.append(color_lab[:, :, 1:] / 128.0)
            
    return np.array(X_train, dtype=np.float32), np.array(Y_train, dtype=np.float32)

from sklearn.model_selection import train_test_split

def train_gan(black_dir, color_dir, epochs=50, batch_size=16):
    X_data, Y_data = load_dataset(black_dir, color_dir)

    # Chia tập train & validation (80% train, 20% val)
    X_train, X_val, Y_train, Y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

    # Xây dựng generator & discriminator
    generator = build_generator()
    discriminator = build_discriminator()

    discriminator.compile(optimizer=Adam(learning_rate=3e-4), loss='binary_crossentropy', metrics=['accuracy'])
    discriminator.trainable = False

    gan_input = Input(shape=(256, 256, 1))
    generated_output = generator(gan_input)
    gan_output = discriminator(generated_output)
    gan = Model(gan_input, gan_output)
    gan.compile(optimizer=Adam(learning_rate=3e-4), loss='binary_crossentropy', metrics=['accuracy'])

    # Gán nhãn cho ảnh thật và giả
    real_labels_train = np.ones((X_train.shape[0], 1))
    fake_labels_train = np.zeros((X_train.shape[0], 1))

    real_labels_val = np.ones((X_val.shape[0], 1))
    fake_labels_val = np.zeros((X_val.shape[0], 1))

    # Huấn luyện discriminator
    fake_images_train = generator.predict(X_train)
    fake_images_val = generator.predict(X_val)

    discriminator.fit(
        np.vstack((Y_train, fake_images_train)),
        np.vstack((real_labels_train, fake_labels_train)),
        validation_data=(
            np.vstack((Y_val, fake_images_val)), 
            np.vstack((real_labels_val, fake_labels_val))
        ),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Huấn luyện GAN
    gan.fit(
        X_train, real_labels_train,
        validation_data=(X_val, real_labels_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1
    )

    # Lưu lại trọng số generator
    generator.save_weights("colorizer_gan_weights.h5")

    # Đánh giá độ chính xác của discriminator trên tập validation
    _, accuracy = discriminator.evaluate(Y_val, np.ones((Y_val.shape[0], 1)), verbose=1)
    print(f"Final Discriminator Accuracy on Validation Set: {accuracy:.4f}")

    return generator



GPU is available and being used.


In [2]:
model = build_discriminator()
model.summary()

/usr/local/lib/python3.10/dist-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 2)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 128, 128, 64)        │           1,216 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu (LeakyReLU)              │ (None, 128, 128, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 64, 64, 128)         │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization                  │ (None, 64, 64, 128)         │             512 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_1 (LeakyReLU)            │ (None, 64, 64, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 32, 32, 256)         │         295,168 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_1                │ (None, 32, 32, 256)         │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ leaky_re_lu_2 (LeakyReLU)            │ (None, 32, 32, 256)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten (Flatten)                    │ (None, 262144)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 1)                   │         262,145 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 633,921 (2.42 MB)

 Trainable params: 633,153 (2.42 MB)

 Non-trainable params: 768 (3.00 KB)

In [3]:
model1=build_generator()
model1.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_1             │ (None, 256, 256, 1)    │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 256, 256, 64)   │            640 │ input_layer_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d             │ (None, 128, 128, 64)   │              0 │ conv2d_3[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 128, 128, 128)  │         73,856 │ max_pooling2d[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_1           │ (None, 64, 64, 128)    │              0 │ conv2d_4[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 64, 64, 256)    │        295,168 │ max_pooling2d_1[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ max_pooling2d_2           │ (None, 32, 32, 256)    │              0 │ conv2d_5[0][0]         │
│ (MaxPooling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 32, 32, 512)    │      1,180,160 │ max_pooling2d_2[0][0]  │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d             │ (None, 64, 64, 512)    │              0 │ conv2d_6[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate (Concatenate) │ (None, 64, 64, 768)    │              0 │ up_sampling2d[0][0],   │
│                           │                        │                │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 64, 64, 256)    │      1,769,728 │ concatenate[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_1           │ (None, 128, 128, 256)  │              0 │ conv2d_7[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ concatenate_1             │ (None, 128, 128, 384)  │              0 │ up_sampling2d_1[0][0], │
│ (Concatenate)             │                        │                │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 128, 128, 128)  │        442,496 │ concatenate_1[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ up_sampling2d_2           │ (None, 256, 256, 128)  │              0 │ conv2d_8[0][0]         │
│ (UpSampling2D)            │                        │                │                        │
├──────────────────────

 Total params: 3,873,858 (14.78 MB)

 Trainable params: 3,873,858 (14.78 MB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
train_gan("/kaggle/input/dat-pro/color",
            "/kaggle/input/dat-pro/gray")


157/157 ━━━━━━━━━━━━━━━━━━━━ 81s 340ms/step
Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:75: UserWarning: The model does not have any trainable weights.
  warnings.warn("The model does not have any trainable weights.")


625/625 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step - accuracy: 0.4975 - loss: 0.7109
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4935 - loss: 0.7109
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4992 - loss: 0.7107
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4949 - loss: 0.7110
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4909 - loss: 0.7111
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.5020 - loss: 0.7106
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.5074 - loss: 0.7103
Epoch 8/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4895 - loss: 0.7111
Epoch 9/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4982 - loss: 0.7107
Epoch 10/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4929 - loss: 0.7110
Epoch 11/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.4902 - loss: 0.7112
Epoch 12/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/ste

In [ ]:
# Evaluate model accuracy
_, accuracy = discriminator.evaluate(Y_train, np.ones((Y_train.shape[0], 1)), verbose=1)
print(f"Final Discriminator Accuracy: {accuracy:.4f}")


In [1]:
def load_trained_model():
    model = build_generator()
    model.load_weights("colorizer_weights.h5")
    return model

def preprocess_image(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    L_channel = image[:, :, 0] / 255.0
    L_channel = np.expand_dims(L_channel, axis=-1)
    return np.expand_dims(L_channel, axis=0)

def postprocess_image(original, predicted):
    predicted = predicted[0] * 128  # Denormalize to original AB range
    original = cv2.cvtColor(original, cv2.COLOR_BGR2LAB)
    original[:, :, 1:] = predicted
    colorized = cv2.cvtColor(original, cv2.COLOR_LAB2BGR)
    return colorized

def colorize_image(image):
    model = load_trained_model()
    L_channel = preprocess_image(image)
    predicted_AB = model.predict(L_channel)
    return postprocess_image(image, predicted_AB)

# # Gradio Interface
# def gradio_interface(image):
#     return colorize_image(image)

# iface = gr.Interface(fn=gradio_interface, inputs="image", outputs="image")
# iface.launch()